In [1]:
!mkdir ~p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [2]:
!kaggle datasets download ayuraj/asl-dataset

Dataset URL: https://www.kaggle.com/datasets/ayuraj/asl-dataset
License(s): CC0-1.0
  0% 0.00/56.9M [00:00<?, ?B/s]
100% 56.9M/56.9M [00:00<00:00, 1.31GB/s]


In [3]:
import zipfile

zip_ref = zipfile.ZipFile('/content/asl-dataset.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [4]:
!rm -rf /content/asl_dataset/asl_dataset


In [5]:
path="/content/asl_dataset"



# Preparing Dataset

In [26]:
import os
import pandas as pd
image_paths=[]
labels=[]

for folder in os.listdir(path):
  for image in os.listdir(os.path.join(path,folder)):
    image_paths.append(os.path.join(path,folder,image))
    labels.append(folder)

data={'image_path':image_paths,'Label':labels}
df=pd.DataFrame(data)
df.to_csv('dataset.csv',index=False)
print("trained csv")

trained csv


In [29]:
df=pd.read_csv("dataset.csv")

df['Label'].unique()

array(['c', 'd', 'q', '4', '1', '9', 'n', 'w', 'j', 'o', '6', 'f', 'u',
       '3', 'z', '5', 'r', '8', 'y', '7', 'l', 's', 'k', '0', 'm', 'b',
       'v', '2', 'x', 'h', 'p', 't', 'e', 'g', 'i', 'a'], dtype=object)

In [31]:
length=len(df['Label'].unique())
length

36

In [33]:
from sklearn.model_selection import train_test_split

train_df,val_df=train_test_split(df,test_size=0.2,random_state=42)


#Data Augmentation

In [35]:

from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen=ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,



)

val_datagen=ImageDataGenerator(rescale=1./255)

In [38]:
# Training data
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='image_path',
    y_col='Label',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',

)

validation_generator = val_datagen.flow_from_dataframe(dataframe=val_df,
                                                        x_col='image_path',
                                                        y_col='Label',
                                                        target_size=(224, 224),
                                                        batch_size=32,
                                                        class_mode='categorical')

Found 2012 validated image filenames belonging to 36 classes.
Found 503 validated image filenames belonging to 36 classes.


## Model Definition:

Using : Inceptionv3
- changing the final layer to a softmax classification layer with classes len.

In [39]:
from keras.applications import InceptionV3

base_model=InceptionV3(weights='imagenet',include_top=False,input_shape=(224,224,3))

In [40]:

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.preprocessing import image


In [45]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(length, activation='softmax')(x)


In [46]:
model=Model(inputs=base_model.input,outputs=predictions)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [48]:
model.fit(train_generator, epochs=5,validation_data=validation_generator)

Epoch 1/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 32s 501ms/step - accuracy: 0.8830 - loss: 0.2852 - val_accuracy: 0.7952 - val_loss: 0.8221
Epoch 2/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 31s 488ms/step - accuracy: 0.8850 - loss: 0.3339 - val_accuracy: 0.5507 - val_loss: 1.8909
Epoch 3/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 33s 518ms/step - accuracy: 0.8865 - loss: 0.3503 - val_accuracy: 0.7197 - val_loss: 0.9304
Epoch 4/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 31s 484ms/step - accuracy: 0.9134 - loss: 0.2555 - val_accuracy: 0.8231 - val_loss: 0.5863
Epoch 5/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 31s 499ms/step - accuracy: 0.9347 - loss: 0.1954 - val_accuracy: 0.7555 - val_loss: 1.0957


In [71]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Load image and resize
img = image.load_img("/content/asl_dataset/3/hand1_3_bot_seg_2_cropped.jpeg", target_size=(224, 224))

# Convert to array
img_array = image.img_to_array(img)

# Normalize
img_array = img_array / 255.0

# Expand dims to simulate batch of 1
img_processed = np.expand_dims(img_array, axis=0)

# Predict
prediction = model.predict(img_processed)
print(x[np.argmax(prediction)])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
3


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os

# List images in the folder
folder_path = '/content/asl_dataset/1'
image_files = os.listdir(folder_path)

# Choose one image (e.g., the first)
image_path ="/content/asl_dataset/1/hand1_1_bot_seg_1_cropped.jpeg"
# Load and show
img = mpimg.imread(image_path)
plt.imshow(img)
plt.axis('off')  # Hide axes
plt.title("Sample from class '1'")
plt.show()


In [68]:
x=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J',
 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T',
 'U', 'V', 'W', 'X', 'Y', 'Z']
